In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [3]:
from synthesizer.inference import Synthesizer
from synthesizer_like_translator.inference import Synthesizer as Translator
from synthesizer.kaldi_interface import KaldiInterface
from encoder import inference as encoder
from encoder import inference_accent as encoder_accent
from vocoder import inference as vocoder
import numpy as np
from pathlib import Path
from utils.argutils import print_args
import random
import IPython.display as ipd
from synthesizer.hparams import hparams


encoder_speaker_weights = Path("/home/grads/q/quamer.waris/projects/Accentron/pretrained_model/pretrained/encoder/saved_models/pretrained.pt")
encoder_accent_weights = Path("/home/grads/q/quamer.waris/projects/Accentron/pretrained_model/pretrained/encoder/saved_models/encoder_accent.pt")
vocoder_weights = Path("/home/grads/q/quamer.waris/projects/Accentron/pretrained_model/pretrained/vocoder/saved_models/pretrained/pretrained.pt")
#syn_dir = Path('/mnt/data1/waris/model_outputs/accentron/parallel_vctk_mic2/logs-Accetron_train_parallel_vctk/taco_pretrained')
syn_dir = Path("/mnt/data1/waris/model_outputs/accentron/parallel/logs-Accetron_train_parallel/taco_pretrained")
#syn_dir_trans = Path("/mnt/data1/waris/model_outputs/translator/sythesizer_like/logs-translator_train/taco_pretrained")
syn_dir_trans = Path("/mnt/data1/waris/model_outputs/translator/sythesizer_like_train_set/logs-translator_train/taco_pretrained")

encoder.load_model(encoder_speaker_weights)
encoder_accent.load_model(encoder_accent_weights)
synthesizer = Synthesizer(syn_dir)
translator = Translator(syn_dir_trans)
vocoder.load_model(vocoder_weights)
#hparams = hparams.parse()

Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "encoder_accent.pt" trained to step 90001
Found synthesizer "Accetron_train_parallel" trained to step 204001
Found synthesizer "translator_train" trained to step 294001
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at /home/grads/q/quamer.waris/projects/Accentron/pretrained_model/pretrained/vocoder/saved_models/pretrained/pretrained.pt


In [4]:
def synthesize(bnf, embed):
    spec = synthesizer.synthesize_spectrograms([bnf], [embed])[0]
    # print(spec.shape)
    generated_wav = vocoder.infer_waveform(spec)
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    return generated_wav

In [5]:
def translate_ppg(bnf, embed):
    spec = translator.synthesize_spectrograms([bnf], [embed])[0]
    return spec

In [6]:
import librosa

def generate_accent_embed(src_utterance_path):
    wav, _ = librosa.load(src_utterance_path, hparams.sample_rate)
    wav = encoder.preprocess_wav(wav)
    embed_accent = encoder_accent.embed_utterance(wav)

    return embed_accent


In [7]:
import librosa

def generate_speaker_embed(tgt_utterance_path):
    wav, _ = librosa.load(tgt_utterance_path, hparams.sample_rate)
    wav = encoder.preprocess_wav(wav)
    embed_speaker = encoder.embed_utterance(wav)

    return embed_speaker

In [7]:
import os
src_speaker = 'Mandarin_mini_testset'
utterance_id = 'MF1_ENG_0001_1'
kaldi_dir = '/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_mini_testset/kaldi'
ki = KaldiInterface(wav_scp=str(os.path.join(kaldi_dir, 'wav.scp')),
                    bnf_scp=str(os.path.join(kaldi_dir, 'bnf/feats.scp')))
bnf = ki.get_feature('_'.join([src_speaker, utterance_id]), 'bnf')

In [8]:
bnf.shape

(516, 256)

In [9]:
src_utterance_path = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/BDL/wav/arctic_b0534.wav'

embed_accent = generate_accent_embed(src_utterance_path)

print(embed_accent.shape)

(256,)


In [10]:
bnf_native = translate_ppg(bnf, embed_accent)

bnf_native.shape

Constructing model: Tacotron
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/legacy_tf_layers/convolutional.py:263: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
  warnings.warn('`tf.layers.conv1d` is deprecated and '
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated an

Tensor("Tacotron_model/strided_slice_1:0", shape=(), dtype=int32, device=/device:CPU:0)
<dtype: 'float32'>
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'Tacotron_model/inference/decoder/assert_greater/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py", line 206, in wrapper
    return target(*args, **kwargs)  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 1047, in assert_greater
    return _binary_assert('>', 'assert_greater', math_ops.greater, np.greater, x,  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 429, in _binary_assert
   

/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/legacy_tf_layers/core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '


initialisation done /gpu:0
Initialized Tacotron model. Dimensions (? = dynamic shape): 
  Train mode:               False
  Eval mode:                False
  GTA mode:                 False
  Synthesis mode:           True
  Input:                    (?, ?, 256)
  device:                   0
  embedding:                (?, ?, 256)
  enc conv out:             (?, ?, 512)
  encoder out (cond):       (?, ?, 768)
  decoder out:              (?, ?, 256)
  residual out:             (?, ?, 1024)
  projected residual out:   (?, ?, 256)
  mel out:                  (?, ?, 256)
  <stop_token> out:         (?, ?)
  Tacotron Parameters       91.878 Million.
Loading checkpoint: /mnt/data1/waris/model_outputs/translator/sythesizer_like/logs-translator_train/taco_pretrained/tacotron_model.ckpt-108001


2022-01-27 11:13:49.223724: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-27 11:13:49.712571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12748 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6


INFO:tensorflow:Restoring parameters from /mnt/data1/waris/model_outputs/translator/sythesizer_like/logs-translator_train/taco_pretrained/tacotron_model.ckpt-108001


2022-01-27 11:13:50.466789: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12748 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6
2022-01-27 11:13:51.256408: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-01-27 11:13:52.206874: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-01-27 11:13:53.129948: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(480, 256)

In [9]:
mel_input, mel_pred, post_mel = translate(bnf, embed_accent)

100%|██████████| 400/400 [00:03<00:00, 107.02it/s]


In [10]:
bnf_native = t.reshape(post_mel, (-1,256))

bnf_native = bnf_native.detach().cpu().numpy()

bnf_native.shape

(400, 256)

In [14]:
import os
src_speaker = 'Mandarin_mini_testset'
utterance_id = 'MF1_ENG_0001_1'
kaldi_dir = '/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_mini_testset/kaldi'
ki = KaldiInterface(wav_scp=str(os.path.join(kaldi_dir, 'wav.scp')),
                    bnf_scp=str(os.path.join(kaldi_dir, 'bnf/feats.scp')))
bnf = ki.get_feature('_'.join([src_speaker, utterance_id]), 'bnf')

In [11]:
#tgt_utterance_path = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/BDL/wav/arctic_b0534.wav'
tgt_utterance_path = '/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_mini_testset/MF1_ENG_0001_1.wav'

embed_speaker = generate_speaker_embed(tgt_utterance_path)

print(embed_speaker.shape)

(256,)


In [12]:
synthesis_wav = synthesize(bnf_native, embed_speaker)
ipd.Audio(synthesis_wav, rate=hparams.sample_rate)

Constructing model: Tacotron


/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/legacy_tf_layers/convolutional.py:263: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
  warnings.warn('`tf.layers.conv1d` is deprecated and '
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated an

Tensor("Tacotron_model/strided_slice_1:0", shape=(), dtype=int32, device=/device:CPU:0)
<dtype: 'float32'>
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'Tacotron_model/inference/decoder/assert_greater/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py", line 206, in wrapper
    return target(*args, **kwargs)  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 1047, in assert_greater
    return _binary_assert('>', 'assert_greater', math_ops.greater, np.greater, x,  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 429, in _binary_assert
   

/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/legacy_tf_layers/core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '


initialisation done /gpu:0
Initialized Tacotron model. Dimensions (? = dynamic shape): 
  Train mode:               False
  Eval mode:                False
  GTA mode:                 False
  Synthesis mode:           True
  Input:                    (?, ?, 256)
  device:                   0
  embedding:                (?, ?, 256)
  enc conv out:             (?, ?, 512)
  encoder out (cond):       (?, ?, 768)
  decoder out:              (?, ?, 80)
  residual out:             (?, ?, 512)
  projected residual out:   (?, ?, 80)
  mel out:                  (?, ?, 80)
  <stop_token> out:         (?, ?)
  Tacotron Parameters       28.653 Million.
Loading checkpoint: /mnt/data1/waris/model_outputs/accentron/parallel/logs-Accetron_train_parallel/taco_pretrained/tacotron_model.ckpt-204001
INFO:tensorflow:Restoring parameters from /mnt/data1/waris/model_outputs/accentron/parallel/logs-Accetron_train_parallel/taco_pretrained/tacotron_model.ckpt-204001


2022-01-27 11:14:18.184770: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12748 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6


{| ████████████████ 85500/86400 | Batch Size: 9 | Gen Rate: 6.6kHz | }

In [13]:
from scipy.io import wavfile

output_dir = '/home/grads/q/quamer.waris/projects/ac-vc/synthesis_output/parallel_report_xxx'
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)
output_file = os.path.join(output_dir, 'man_man_f1_222.wav')
wavfile.write(output_file, hparams.sample_rate, synthesis_wav)


In [7]:
import os
from scipy.io import wavfile
import numpy as np


# utterance_ids = ['arctic_b0534', 'arctic_b0537', 'arctic_b0538', 'arctic_b0539']
# speakers = ['BDL', 'NJS', 'TXHC', 'YKWK', 'ZHAA']
utterance_ids = ['_001_mic2', '_002_mic2', '_003_mic2', '_004_mic2']
speakers = ['s5', 'p304', 'p250', 'p252', 'p267']

f_path = '/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/'

for speaker in speakers:
    for utterance_id in utterance_ids:
        kaldi_dir = f_path + speaker +'/kaldi'
        utterance_idx = speaker+utterance_id
        #kaldi_dir = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/'+speaker+'/kaldi'
        ki = KaldiInterface(wav_scp=str(os.path.join(kaldi_dir, 'wav.scp')),
                            bnf_scp=str(os.path.join(kaldi_dir, 'bnf/feats.scp')))
        bnf = ki.get_feature('_'.join([speaker, utterance_idx]), 'bnf')

        for speaker_target in speakers:

            if speaker_target == speaker:
                continue
            
            utterance_idy = speaker_target+utterance_id
            tgt_utterance_path = f_path+str(speaker_target)+'/wav/'+str(utterance_idy)+'.wav'
            embed_speaker = generate_speaker_embed(tgt_utterance_path)

            synthesis_wav = synthesize(bnf, embed_speaker)

            output_dir = '/home/grads/q/quamer.waris/projects/Accentron/synthesis_output/parallel_shalini_vctk_mic2_model_100k/'+str(speaker)+'_'+str(speaker_target)
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            filename = str(utterance_id)+'.wav'
            output_file = os.path.join(output_dir, filename)
            wavfile.write(output_file, hparams.sample_rate, synthesis_wav)


Constructing model: Tacotron
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/legacy_tf_layers/convolutional.py:263: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
  warnings.warn('`tf.layers.conv1d` is deprecated and '
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated an

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Tensor("Tacotron_model/strided_slice_1:0", shape=(), dtype=int32, device=/device:CPU:0)
<dtype: 'float32'>
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'Tacotron_model/inference/decoder/assert_greater/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py", line 206, in wrapper
    return target(*args, **kwargs)  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 1047, in assert_greater
    return _binary_assert('>', 'assert_greater', math_ops.greater, np.greater, x,  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 429, in _binary_assert
   

...

initialisation done /gpu:0
Initialized Tacotron model. Dimensions (? = dynamic shape): 
  Train mode:               False
  Eval mode:                False
  GTA mode:                 False
  Synthesis mode:           True
  Input:                    (?, ?, 256)
  device:                   0
  embedding:                (?, ?, 256)
  enc conv out:             (?, ?, 512)
  encoder out (cond):       (?, ?, 768)
  decoder out:              (?, ?, 80)
  residual out:             (?, ?, 512)
  projected residual out:   (?, ?, 80)
  mel out:                  (?, ?, 80)
  <stop_token> out:         (?, ?)
  Tacotron Parameters       28.653 Million.
Loading checkpoint: /mnt/data1/waris/model_outputs/accentron/parallel_vctk_mic2/logs-Accetron_train_parallel_vctk/taco_pretrained/tacotron_model.ckpt-100001


2022-02-10 17:46:15.069653: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-10 17:46:15.749074: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20267 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6


INFO:tensorflow:Restoring parameters from /mnt/data1/waris/model_outputs/accentron/parallel_vctk_mic2/logs-Accetron_train_parallel_vctk/taco_pretrained/tacotron_model.ckpt-100001


2022-02-10 17:46:16.773123: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20267 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6
2022-02-10 17:46:17.773240: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-02-10 17:46:18.684533: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-02-10 17:46:20.220209: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


{| ████████████████ 76000/76800 | Batch Size: 8 | Gen Rate: 6.5kHz | } | }

In [7]:
import os
from scipy.io import wavfile
import numpy as np


utterance_ids = ['arctic_b0534', 'arctic_b0537', 'arctic_b0538', 'arctic_b0539']
speakers = ['BDL', 'NJS', 'TXHC', 'YKWK', 'ZHAA']

for speaker in speakers:
    for utterance_id in utterance_ids:
        kaldi_dir = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/'+speaker+'/kaldi'
        ki = KaldiInterface(wav_scp=str(os.path.join(kaldi_dir, 'wav.scp')),
                            bnf_scp=str(os.path.join(kaldi_dir, 'bnf/feats.scp')))
        bnf = ki.get_feature('_'.join([speaker, utterance_id]), 'bnf')

        acc_utterance_path = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/BDL/wav/'+str(utterance_id)+'.wav'
        embed_accent = generate_accent_embed(acc_utterance_path)
        bnf_native = translate_ppg(bnf, embed_accent)

        tgt_utterance_path = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/'+str(speaker)+'/wav/'+str(utterance_id)+'.wav'
        embed_speaker = generate_speaker_embed(tgt_utterance_path)

        synthesis_wav = synthesize(bnf_native, embed_speaker)

        output_dir = '/home/grads/q/quamer.waris/projects/ac-vc/synthesis_output/train_set/ppg_to_ppg_250k/'+str(speaker)
        if not os.path.isdir(output_dir):
            os.makedirs(output_dir)
        filename = str(utterance_id)+'.wav'
        output_file = os.path.join(output_dir, filename)
        wavfile.write(output_file, hparams.sample_rate, synthesis_wav)
        # wavfile.write(output_file, hparams.sample_rate, synthesis_wav.astype(np.int16))


Constructing model: Tacotron
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/legacy_tf_layers/convolutional.py:263: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
  warnings.warn('`tf.layers.conv1d` is deprecated and '
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated an

Tensor("Tacotron_model/strided_slice_1:0", shape=(), dtype=int32, device=/device:CPU:0)
<dtype: 'float32'>
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'Tacotron_model/inference/decoder/assert_greater/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py", line 206, in wrapper
    return target(*args, **kwargs)  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 1047, in assert_greater
    return _binary_assert('>', 'assert_greater', math_ops.greater, np.greater, x,  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 429, in _binary_assert
   

/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/keras/legacy_tf_layers/core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '


initialisation done /gpu:0
Initialized Tacotron model. Dimensions (? = dynamic shape): 
  Train mode:               False
  Eval mode:                False
  GTA mode:                 False
  Synthesis mode:           True
  Input:                    (?, ?, 256)
  device:                   0
  embedding:                (?, ?, 256)
  enc conv out:             (?, ?, 512)
  encoder out (cond):       (?, ?, 768)
  decoder out:              (?, ?, 256)
  residual out:             (?, ?, 1024)
  projected residual out:   (?, ?, 256)
  mel out:                  (?, ?, 256)
  <stop_token> out:         (?, ?)
  Tacotron Parameters       91.878 Million.
Loading checkpoint: /mnt/data1/waris/model_outputs/translator/sythesizer_like_train_set/logs-translator_train/taco_pretrained/tacotron_model.ckpt-254001


...

INFO:tensorflow:Restoring parameters from /mnt/data1/waris/model_outputs/translator/sythesizer_like_train_set/logs-translator_train/taco_pretrained/tacotron_model.ckpt-254001


2022-02-02 10:47:11.794317: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4422 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6
2022-02-02 10:47:13.537060: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-02-02 10:47:16.100052: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-02 10:47:18.330304: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Constructing model: Tacotron
Tensor("Tacotron_model/strided_slice_1:0", shape=(), dtype=int32, device=/device:CPU:0)
<dtype: 'float32'>
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'Tacotron_model/inference/decoder/assert_greater/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py", line 206, in wrapper
    return target(*args, **kwargs)  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", line 1047, in assert_greater
    return _binary_assert('>', 'assert_greater', math_ops.greater, np.greater, x,  File "/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/tensorflow/python/ops/check_ops.py", li

2022-02-02 10:47:20.764514: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4422 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:25:00.0, compute capability: 8.6


{| ████████████████ 47500/48000 | Batch Size: 5 | Gen Rate: 3.1kHz | }

In [13]:
import os
from scipy.io import wavfile
import numpy as np

#b500, b503, b531
utterance_id = 'arctic_b0503'
speaker = 'TXHC'


kaldi_dir = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/'+speaker+'/kaldi'
ki = KaldiInterface(wav_scp=str(os.path.join(kaldi_dir, 'wav.scp')),
                    bnf_scp=str(os.path.join(kaldi_dir, 'bnf/feats.scp')))
bnf = ki.get_feature('_'.join([speaker, utterance_id]), 'bnf')

acc_utterance_path = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/BDL/wav/'+str(utterance_id)+'.wav'
embed_accent = generate_accent_embed(acc_utterance_path)
bnf_native = translate_ppg(bnf, embed_accent)

tgt_utterance_path = '/mnt/data1/waris/datasets/data/arctic_dataset/all_data/'+str(speaker)+'/wav/'+str(utterance_id)+'.wav'
embed_speaker = generate_speaker_embed(tgt_utterance_path)

synthesis_wav = synthesize(bnf_native, embed_speaker)

output_dir = '/mnt/data1/waris/model_results/translator_synthesizer/ppg_to_ppg_294k/'+str(speaker)
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)
filename = str(utterance_id)+'.wav'
output_file = os.path.join(output_dir, filename)
wavfile.write(output_file, hparams.sample_rate, synthesis_wav)


{| ████████████████ 437000/441600 | Batch Size: 46 | Gen Rate: 43.3kHz | }